In [1]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = 'drive/MyDrive/github_repos/sainfoin_seed_RCNN'
%cd -q $BASE_DIR

Mounted at /content/drive


In [2]:
!pip install -r colab_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 10.3 MB/s eta 0:00:00


In [3]:
import os
import sys
import cv2
import torch
import torchvision
import torchmetrics
import yaml
import time
import logging
import re
import itertools

import pandas as pd
import numpy as np
import albumentations as A

from glob import glob
from pathlib import Path
from tqdm import tqdm
from random import randint
from albumentations.pytorch import ToTensorV2
from torchmetrics.classification import MulticlassAccuracy, MulticlassJaccardIndex
from torchmetrics.detection import IntersectionOverUnion, MeanAveragePrecision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import ToTensor
from torchvision.ops import nms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [4]:
# sys.path.append('./src')
# from src.config import parse_config
from src.utils.loggers import create_logger
from src.model.model import create_model
from src.utils.transforms import train_transforms, val_transforms, collate_fn, inf_transforms
from src.utils.dataset import SeedDataset, dir_sampler
# from data_splitting import train, val, test
# from src.model.train_val import train_model
from src.config import train_dir, val_dir, test_dir, annotation_path, chkpt_dir, tensorboard_dir, log_dir, inference_dir
from src.config import device, cores, classes, n_classes, resize_to, n_epochs, batch_size
from src.config import base_name, lr, momentum, gamma


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [16]:
# Set variables
CHKPT_PATH = 'model_chkpt/frcnn_sainfoin_1.0/frcnn_sainfoin_1.0_100.pth'
IMG_DIR = 'data/images/test'

# Transforms
transforms = inf_transforms()

# Create FasterRCNN model and load trained weights
model = create_model(n_classes, n_obj_det=500)
model.load_state_dict(
  torch.load(
    CHKPT_PATH,
    map_location=device
  )
)
model.eval()
model.to(device)

file_names = glob("*.jpg", root_dir='data/images/test')
for file_name in file_names:
    img_path = Path(IMG_DIR) / file_name
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
    img /= 255.0
    img = [transforms(image=img)['image'].to(device)]

    output = model(img)
    print(output)
    # img = np.transpose(img, (2, 0, 1)).astype(np.float32)
    # img = torch.tensor(img, dtype=torch.float).cuda()
    # img = torch.unsqueeze(img, 0)

# for size in sizes:
  # print(size)
  # path = [re.findall(f'.*{size}.*', point) for point in checkpoints]
  # path = list(itertools.chain.from_iterable(path))[0]
  # print(path)

  # create model backbone
  # model = create_model(n_classes, n_obj_det=500)
  # model.load_state_dict(
  #     torch.load(
  #         './model_chkpt/model49_all_anchors.pth',
  #         map_location=device
  #     ))
  # model.eval()
  # model.to(device)

  # dir_path = os.path.join(inference_dir, f'size_{size}')
  # dir_path = os.path.join(inference_dir, 'old_model')
  # if not os.path.exists(dir_path):
  #   os.mkdir(dir_path)

  # filenames = glob(test_dir + '/*')

  # filenames = [i for i in filenames if i.lower().endswith('.jpg')]
  # print(filenames)
  # for file in filenames:
  #   print(file)

    # img = cv2.imread(file)
    # orig = img.copy()
    # img = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB).astype(np.float32)
    # img /= 255.0
    # img = np.transpose(img, (2, 0, 1)).astype(np.float32)
    # img = torch.tensor(img, dtype=torch.float).cuda()
    # img = torch.unsqueeze(img, 0)
    # print(img.shape)
    # print(type(img))
    # with torch.no_grad():
    #   outputs = model(img)
    # outputs = [{k: v.to('cpu') for k, v in t.items()} for t in outputs]
    # print(len(outputs[0]['boxes']))
    # nms_idx = nms(boxes=outputs[0]['boxes'], scores=outputs[0]['scores'], iou_threshold=.4)
    # print(nms_idx)
    # outputs[0]['boxes'] = outputs[0]['boxes'][nms_idx]
    # outputs[0]['labels'] = outputs[0]['labels'][nms_idx]
    # outputs[0]['scores'] = outputs[0]['scores'][nms_idx]
    # print(len(outputs[0]['boxes']))

    # if len(outputs[0]['boxes']) != 0:
    #   boxes = outputs[0]['boxes'].data.numpy().astype(np.int32)
    #   scores = outputs[0]['scores'].data.numpy()
    #   # boxes = boxes[scores >= detection_threshold].astype(np.int32)
    #   # scores = scores[scores >= detection_threshold]
    #   draw_boxes = boxes.copy()
    #   # get all the predicited class names
    #   pred_classes = [classes[f'{i}'] for i in outputs[0]['labels'].cpu().numpy()]
    #   for j, box in enumerate(draw_boxes):
    #     cv2.rectangle(orig,
    #                   (int(box[0]), int(box[1])),
    #                   (int(box[2]), int(box[3])),
    #                   (255, 255, 0),
    #                   1)
    #     # print(np.round(scores[j], 2))
    #     cv2.putText(orig, pred_classes[j] + f" {np.round(scores[j], 4): .4f}",
    #                         (int(box[0]), int(box[1] - 5)),
    #                         cv2.FONT_HERSHEY_SIMPLEX, .3, (0, 150, 255),
    #                         1, lineType=cv2.LINE_AA)

    #   new_img_path = os.path.join(dir_path, f"{file.split('/')[-1][:-4]}.jpg")
    #   print(new_img_path)

    #   cv2.imwrite(new_img_path, orig)


[{'boxes': tensor([[4162.5254, 2591.3081, 4224.4458, 2637.2959],
        [4187.4775, 3031.7495, 4250.4990, 3081.1855],
        [5615.1323, 3503.2224, 5685.4272, 3552.2646],
        ...,
        [2912.4976, 2261.4963, 2942.5312, 2287.7617],
        [3503.5339, 3528.2954, 3531.1003, 3558.5103],
        [5703.0815, 2151.0923, 5726.1572, 2177.1868]],
       grad_fn=<StackBackward0>), 'labels': tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 2, 1, 1, 3, 1, 2, 3,
        2, 3, 3, 3, 2, 3, 3, 2, 3, 1, 3, 2, 2, 1, 3, 2, 2, 3, 2, 1, 1, 1, 2, 1,
        1, 1, 3, 3, 3, 2, 2, 2, 2, 3, 1, 2, 1, 2, 2, 3, 1, 3, 1, 1, 2, 2, 2, 1,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 

KeyboardInterrupt: 

In [ ]:
device

'cuda'

In [5]:
# Read in all of the image metadata
img_data = pd.read_csv('./data/power_analysis/results.csv')
print(img_data.head())

train_imgs = img_data.loc[img_data['class']=='train']
val_imgs = img_data.loc[img_data['class']=='val']

train_dataset = SeedDataset(image_dir=train_dir,
                            annot_dir=annotation_dir,
                            resize_dims=(resize_to, resize_to),
                            classes=classes,
                            transforms=val_transforms(),
                            subset=list(train_imgs['img_name'].unique()))

val_dataset = SeedDataset(image_dir=val_dir,
                            annot_dir=annotation_dir,
                            resize_dims=(resize_to, resize_to),
                            classes=classes,
                            transforms=val_transforms(),
                            subset=list(val_imgs['img_name'].unique()))

train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

   Unnamed: 0 img_id   img_name  class                variety     method  \
0           0  12600  12600.jpg  train  Rocky Mountain Remont  HLDP7x35s   
1           1  9adc6  9adc6.JPG  train               Shoshone       BT3X   
2           2  1cd49  1cd49.jpg  train       AAC Mountainview  HLDP7x35s   
3           3  6eb3d  6eb3d.JPG  train               Shoshone       BT3X   
4           4  7b24d  7b24d.JPG  train               Shoshone       BT3X   

   sample_mass_g  rep  legume_fruit_mass_g  legume_seed_mass_g comments  
0              2    7               2.0189               1.569      NaN  
1              2    5               2.0131               1.677      NaN  
2              4    1               4.0014               2.921      NaN  
3              1   10               1.0075               0.887      NaN  
4              3    8               3.0066               2.736      NaN  


In [ ]:
detection_threshold = 0.8
sizes = ['0.05', '0.1', '0.2', '0.5', '1.0']
chkpt_paths = ['./model_chkpt/sainfoin_seed_0.05/sainfoin_seed_0.05_50.pth',
               './model_chkpt/sainfoin_seed_0.1/sainfoin_seed_0.1_50.pth',
               './model_chkpt/sainfoin_seed_0.2/sainfoin_seed_0.2_50.pth',
               './model_chkpt/sainfoin_seed_0.5/sainfoin_seed_0.5_50.pth',
               './model_chkpt/sainfoin_seed_1.0/sainfoin_seed_1.0_50.pth',]
# sizes = ['1.0']
# checkpoints = glob('./model_chkpt/*')
# print(chkpt_paths)
for path in chkpt_paths:
  print(path)
  size = sizes[chkpt_paths.index(path)]
  print(size)
  dir_path = os.path.join(inference_dir, f'size_{size}')
  if not os.path.exists(dir_path):
    os.mkdir(dir_path)
  # path = [re.findall(f'.*{size}.*', point) for point in checkpoints]
  # path = list(itertools.chain.from_iterable(path))[0]
  # print(path)

  # create model backbone
  model = create_model(n_classes, n_obj_det=500)
  model.load_state_dict(
      torch.load(
          path,
          map_location=device
      ))
  model.eval()
  model.to(device)

  train_iter = iter(train_loader)
  val_iter = iter(val_loader)
  while True:
    try:
      img, targets = next(val_iter)
      img = img[0]

      img_id = targets[0]['img_name'][:-4]

      print(img_id)
      orig = np.transpose(img.clone().cpu().numpy(), (1, 2, 0))*255
      orig = cv2.cvtColor(orig, cv2.COLOR_RGB2BGR)

      # print(targets[0]['labels'])
      img = torch.unsqueeze(img, 0).to(device)
      with torch.no_grad():
        outputs = model(img)
      # print(outputs[0]['labels'])
      metric = IntersectionOverUnion(class_metrics=True, respect_labels=True).to(device)
      # # MAP = MeanAveragePrecision(iou_type='bbox')
      # iou = metric(preds=outputs, target=list(targets))
      # # map_res = MAP(preds=outputs, target=list(targets))
      # print(iou)

      # print(len(outputs[0]['boxes']))
      nms_idx = nms(boxes=outputs[0]['boxes'], scores=outputs[0]['scores'], iou_threshold=.5)
      # print(nms_idx)
      outputs[0]['boxes'] = outputs[0]['boxes'][nms_idx]
      outputs[0]['labels'] = outputs[0]['labels'][nms_idx]
      outputs[0]['scores'] = outputs[0]['scores'][nms_idx]
      # print(len(outputs[0]['boxes']))

      if len(outputs[0]['boxes']) != 0:
        boxes = outputs[0]['boxes'].data.cpu().numpy().astype(np.int32)
        scores = outputs[0]['scores'].data.cpu().numpy()
        labels = outputs[0]['labels'].data.cpu().numpy()
        boxes = boxes[scores >= detection_threshold].astype(np.int32)
        labels = labels[scores >= detection_threshold]
        scores = scores[scores >= detection_threshold]

        # metric = IntersectionOverUnion(class_metrics=True, respect_labels=True).to(device)
        metric = MeanAveragePrecision(iou_type='bbox',
                                      max_detection_thresholds=[1, 10, 100, 1000],
                                      class_metrics=True).to(device)
        adj_outputs = [{
            'boxes': torch.Tensor(boxes).to(device),
            'labels': torch.Tensor(labels).int().to(device),
            'scores': torch.Tensor(scores).to(device)
        }]

        # for k, v in targets[0].items():
        #   if k !='img_name':
        #     print(k, v.to(device))
        adj_targets = [{k: v.to(device) for k, v in targets[0].items() if k != 'img_name'}]
        # print(f"Targets: {adj_targets}")
        iou = metric(preds=adj_outputs, target=adj_targets)
        print(iou)
        # print(f"Outputs: {adj_outputs}")


        draw_boxes = boxes.copy()
        # get all the predicited class names
        pred_classes = [classes[f'{i}'] for i in labels]

        # Draw ground truth boxes
        gt_labels = targets[0]['labels']
        gt_boxes = targets[0]['boxes']
        for j, box in enumerate(gt_boxes):
          # print(box)
          box = box.cpu().numpy()
          # print(box)
          cv2.rectangle(orig,
                        (int(box[0]), int(box[1])),
                        (int(box[2]), int(box[3])),
                        (0, 150, 0),
                        2)

        # Draw predictions and scores
        for j, box in enumerate(draw_boxes):
          cv2.rectangle(orig,
                        (int(box[0]), int(box[1])),
                        (int(box[2]), int(box[3])),
                        (0, 0, 255),
                        2)
          # print(np.round(scores[j], 2))
          cv2.putText(orig, pred_classes[j] + f" {np.round(scores[j], 4): .4f}",
                              (int(box[0]), int(box[1] - 5)),
                              cv2.FONT_HERSHEY_SIMPLEX, .4, (0, 150, 255),
                              1, lineType=cv2.LINE_AA)



      new_img_path = os.path.join(dir_path, f"{img_id}.jpg")
      print(new_img_path)

      cv2.imwrite(new_img_path, orig)




    except StopIteration:
      print("end")
      break



./model_chkpt/sainfoin_seed_0.05/sainfoin_seed_0.05_50.pth
0.05
013b9
{'map': tensor(0.3833), 'map_50': tensor(0.6202), 'map_75': tensor(0.4620), 'map_small': tensor(0.7406), 'map_medium': tensor(0.3642), 'map_large': tensor(-1.), 'mar_1': tensor(0.0156), 'mar_10': tensor(0.1969), 'mar_100': tensor(0.4298), 'mar_small': tensor(0.7600), 'mar_medium': tensor(0.4153), 'mar_large': tensor(-1.), 'map_per_class': tensor([0.0000, 0.5544, 0.5954]), 'mar_100_per_class': tensor([0.0000, 0.6300, 0.6594]), 'classes': tensor([1, 2, 3], dtype=torch.int32)}
./inference/size_0.05/013b9.jpg
0f6c9
{'map': tensor(0.6785), 'map_50': tensor(0.9752), 'map_75': tensor(0.9543), 'map_small': tensor(0.6673), 'map_medium': tensor(0.6789), 'map_large': tensor(-1.), 'mar_1': tensor(0.0311), 'mar_10': tensor(0.3292), 'mar_100': tensor(0.7186), 'mar_small': tensor(0.6667), 'mar_medium': tensor(0.7214), 'mar_large': tensor(-1.), 'map_per_class': tensor([0.6432, 0.7137]), 'mar_100_per_class': tensor([0.6826, 0.7545]),

In [ ]:
preds = [
    {
        "boxes": torch.tensor([[150, 200, 250, 300], [320, 350, 470, 475]]),
        "scores": torch.tensor([0.99, 0.99]),
        "labels": torch.tensor([4, 5])
        }
    ]
target = [
    {
        "boxes": torch.tensor([[320, 350, 470, 475], [150, 200, 250, 300], [204, 214, 256, 267]]),
        "labels": torch.tensor([5, 4, 3]),
        }
    ]

print(preds)
print(target)
metric = MeanAveragePrecision(iou_type='bbox',
                              class_metrics=True,
                              max_detection_thresholds=[1, 10, 100, 1000])
metric(preds, target)
# {'iou': tensor(0.4307)}

[{'boxes': tensor([[150, 200, 250, 300],
        [320, 350, 470, 475]]), 'scores': tensor([0.9900, 0.9900]), 'labels': tensor([4, 5])}]
[{'boxes': tensor([[320, 350, 470, 475],
        [150, 200, 250, 300],
        [204, 214, 256, 267]]), 'labels': tensor([5, 4, 3])}]


{'map': tensor(0.6667),
 'map_50': tensor(0.6667),
 'map_75': tensor(0.6667),
 'map_small': tensor(-1.),
 'map_medium': tensor(0.),
 'map_large': tensor(1.),
 'mar_1': tensor(0.6667),
 'mar_10': tensor(0.6667),
 'mar_100': tensor(0.6667),
 'mar_small': tensor(-1.),
 'mar_medium': tensor(0.),
 'mar_large': tensor(1.),
 'map_per_class': tensor([0., 1., 1.]),
 'mar_100_per_class': tensor([0., 1., 1.]),
 'classes': tensor([3, 4, 5], dtype=torch.int32)}

In [ ]:
metric = MeanAveragePrecision(iou_type='bbox',
                              class_metrics=True,
                              max_detection_thresholds=[1, 10, 100, 1000])

metric(preds=adj_outputs,
       target=adj_targets)

{'map': tensor(0.3833),
 'map_50': tensor(0.6202),
 'map_75': tensor(0.4620),
 'map_small': tensor(0.7406),
 'map_medium': tensor(0.3642),
 'map_large': tensor(-1.),
 'mar_1': tensor(0.0156),
 'mar_10': tensor(0.1969),
 'mar_100': tensor(0.4298),
 'mar_small': tensor(0.7600),
 'mar_medium': tensor(0.4153),
 'mar_large': tensor(-1.),
 'map_per_class': tensor([0.0000, 0.5544, 0.5954]),
 'mar_100_per_class': tensor([0.0000, 0.6300, 0.6594]),
 'classes': tensor([1, 2, 3], dtype=torch.int32)}